## HW2: Master info

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')
import sys
from bs4 import BeautifulSoup as BSoup
import requests
formLink = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247"
showLink = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247"
r = requests.get(formLink)
soup = BSoup(r.text, "lxml")

In [ ]:
def showLinkGen(htmlP, htmlV, fieldP, fieldV, yearP, yearV, semP, semV):
  return "{}&{}={}&{}={}&{}={}&{}={}".format(showLink, htmlP, htmlV, fieldP, fieldV, yearP, yearV, semP, semV)

In [ ]:
selectFields = soup.find_all("select")
infoField=selectFields[0].find("option", text="Informatique")["value"]
infoFieldParam=selectFields[0]["name"]
semParam=selectFields[2]["name"]
msSem1_text="Master semestre 1"
msSemPF_text="Projet Master automne"
msSemPS_text="Projet Master printemps"
msSem1=selectFields[2].find("option", text=msSem1_text)["value"]
msSemPF=selectFields[2].find("option", text=msSemPF_text)["value"]
msSemPS=selectFields[2].find("option", text=msSemPS_text)["value"]
allYears = [y["value"] for y in selectFields[1].find_all("option")[1:]]
yearParam = selectFields[1]["name"]
htmlradiobutton=soup.find("input", type="radio")
def get_html(sem, year):
    link = showLinkGen(htmlradiobutton["name"], htmlradiobutton["value"], infoFieldParam, infoField, yearParam, 
                 year, semParam, sem)
    return requests.get(link)

In [ ]:
def get_ms_dataframe(request_link):
  soup2 = BSoup(request_link.text, "lxml")
  elems=soup2.find_all("tr")[2:]
  titleinfo = soup2.find("font")
  if(titleinfo is None):
      return pd.DataFrame([])
  else:
      titleinfo = titleinfo.text.split(', ')
      semester=titleinfo[2]
      semester=1 if semester==msSem1_text else (2 if semester==msSemPF_text else 3)
      year=titleinfo[1]
      all_data=[]
      for elem in elems:
        items=elem.find_all("td")
        gender = "M" if (items[0].text == "Monsieur") else "F"
        sciper = int(items[10].text)
        spec = items[4].text != ""
        minor = items[6].text != ""
        all_data.append({"Scipper": sciper, "Sex": gender, "Year": year, "Semester": semester,
                        "Minor": minor, "Specialization": spec})
      return pd.DataFrame(all_data)

In [ ]:
def get_ms_alldata():
  all_data = []
  for sem in [msSem1, msSemPF, msSemPS]:
    for year in allYears:
        all_data.append(get_ms_dataframe(get_html(sem, year)))
  return pd.concat(all_data)
ms_data = get_ms_alldata()
ms_data

In [ ]:
ms_data.reset_index(None,drop=True,inplace=True)
ms_data.dtypes

In [ ]:
master_semester1 = ms_data[ms_data.Semester == 1]
year = master_semester1['Year'].str.split('-',expand=True)
master_semester1.Year = year[0].astype(int)
master_semester1

In [ ]:
master_project = ms_data[ms_data.Semester != 1]
year = master_project['Year'].str.split('-',expand=True)
master_project.Year = np.where(master_project['Semester']==2, year[0].astype(int), year[1].astype(int))
master_project

In [ ]:
master_semester1.Scipper.is_unique

In [ ]:
master_semester1 = master_semester1.sort_values("Year")
master_semester1 = master_semester1.drop_duplicates("Scipper", keep='first')
master_semester1.rename(columns = {'Year':'FirstYear'}, inplace=True)
master_semester1.drop('Specialization',axis=1, inplace=True)
master_semester1.drop('Minor',axis=1, inplace=True)
master_semester1.drop('Semester', axis=1, inplace=True)
master_semester1

In [ ]:
master_project.Scipper.is_unique

In [ ]:
master_project = master_project.sort_values("Year")
master_project = master_project.drop_duplicates("Scipper", keep='last')
master_project.rename(columns = {'Year':'LastYear'}, inplace=True)
master_project.Scipper.is_unique

In [ ]:
master_semester1.head()

In [ ]:
master_project.head()

In [ ]:
ms = pd.merge(master_semester1,master_project, how='inner')
ms

In [ ]:
ms['Staytime'] = (ms.LastYear - ms.FirstYear)*12 + (ms.Semester-1)*6
ms

In [ ]:
ms_sex = ms.groupby('Sex')
ms_sex['Staytime'].mean()

In [ ]:
ms_spec = ms.groupby('Specialization')
ms_spec['Staytime'].mean()

In [ ]:
ms_minor = ms.groupby('Minor')
ms_minor['Staytime'].mean()

In [ ]:
ms_minor_spec = ms.groupby(['Minor','Specialization'])
ms_minor_spec['Staytime'].mean()

In [ ]:
ms_minor_spec_sex = ms.groupby(['Sex','Minor','Specialization'])
ms_minor_spec_sex['Staytime'].mean()